# Conv1d_Autoencoder

In [1]:
import pickle
import os
import sys
import glob
########################################################################


########################################################################
# import additional python-library
########################################################################

import numpy as np
import pandas as pd

import librosa
import librosa.core
import librosa.feature
import librosa.display
import yaml
import pathlib
import optuna
import IPython.display as ipd
import matplotlib.pyplot as plt
import scipy
import warnings
import ipywidgets as widgets

# from import
from tqdm import tqdm
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from keras.models import Model
from keras.layers import Input, Dense
from ipywidgets import interact, interact_manual

from dataset import Mimii_due, Toyadmos
import preprocess as preproc

warnings.filterwarnings('ignore')

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.autograd import Variable
from torch.optim import Adam

## 1D Feature representation - amplitude values

### Dataset MIMII_DUE

In [3]:
# train data dir
target_dir = r'datasets\MIMII_DUE\dev_data\gearbox'
section_name = 'section_00'
dir_name_train = r'\train'
# target_test data dir
dir_name_test = r'\target_test'

In [4]:
dataset_train_mimii_ts = Mimii_due(target_dir, section_name, dir_name_train, extraction_type='amplitude')
dataset_test_mimii_ts = Mimii_due(target_dir, section_name, dir_name_test, extraction_type='amplitude')

100%|████████████████████████████████████████████████████████████████████████████████| 204/204 [00:07<00:00, 29.03it/s]


In [5]:
X_train_mimii_ts, y_train_mimii_ts = dataset_train_mimii_ts.data, dataset_train_mimii_ts.labels
X_test_mimii_ts, y_test_mimii_ts = dataset_test_mimii_ts.data, dataset_test_mimii_ts.labels
X_train_mimii_ts.shape, X_test_mimii_ts.shape, y_train_mimii_ts.shape, y_test_mimii_ts.shape

((1004, 160000), (204, 160000), (1004,), (204,))

In [6]:
# Anomaly data ratio
contamination_mimii = np.round(y_test_mimii_ts.sum() / y_test_mimii_ts.shape, 2)
print(f'Mimii_due_anomaly ratio = {contamination_mimii[0]}')
contamination_mimii = 0.4

Mimii_due_anomaly ratio = 0.53


### Dataset ToyAdmos2

In [7]:
# data dir
target_dir_toyadm = r'datasets\ToyAdmos2'
dir_name_toyadm_anomaly = r'\toyad2_car_A_anomaly'
dir_name_toyadm_normal = r'\toyad2_car_A1_normal'

In [8]:
dataset_toy_ts = Toyadmos(target_dir_toyadm, dir_name_toyadm_normal, dir_name_toyadm_anomaly, extraction_type='amplitude')


100%|██████████████████████████████████████████████████████████████████████████████| 3545/3545 [11:33<00:00,  5.11it/s]


In [9]:
# Train - test - val stratified split
X_train_toy_ts, X_test_toy_ts, X_val_toy_ts, y_train_toy_ts, y_test_toy_ts, y_val_toy_ts = preproc.mix_data(
    [dataset_toy_ts.data], [dataset_toy_ts.labels])
X_train_toy_ts.shape, X_test_toy_ts.shape, X_val_toy_ts.shape, y_train_toy_ts.shape, y_test_toy_ts.shape, y_val_toy_ts.shape

((2871, 192000), (319, 192000), (355, 192000), (2871,), (319,), (355,))

In [10]:
# Anomaly data ratio
contamination_toy = np.round(y_test_toy_ts.sum() / y_test_toy_ts.shape, 2)
print(f'ToyAdmos anomaly ratio = {contamination_toy[0]}')

ToyAdmos anomaly ratio = 0.23


In [11]:
torch.cuda.is_available()

True

In [12]:
def dataloader(X_train, y_train, X_test, y_test, batch_size):
    train_dataset = TensorDataset(torch.tensor(X_train.astype(
        np.float32)), torch.tensor(y_train.astype(np.long)))
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True)
    test_dataset = TensorDataset(
        torch.tensor(X_test.astype(np.float32)),
        torch.tensor(y_test.astype(np.long)))
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [13]:
class Autoencoder(nn.Module):
    '''
    https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/conv_autoencoder.py
    '''
    def __init__(self, input_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 8, 3, stride=1, padding=1), # output_size = input_size
            nn.MaxPool1d(4,4), # output_size = input_size / 4
            nn.ReLU(True),
            nn.Conv1d(8, 16, 3, stride=1, padding=1),  # output_size = input_size / 4 
            nn.MaxPool1d(4,4),  # output_size = input_size / 16
            nn.BatchNorm1d(16),
            nn.ReLU(True),
            nn.Conv1d(16, 32, 3, stride=1, padding=1),  # output_size = input_size /16
            nn.MaxPool1d(4,4),  # output_size = input_size / 64
            nn.ReLU(True),
            nn.Flatten(start_dim=1),
           
            nn.Linear(int((input_size/64)*32), 128),
            nn.ReLU(True),
            nn.Linear(128, 100)
            
            
        )
        self.decoder = nn.Sequential(
            nn.Linear(100, 128),
            nn.ReLU(True),
            nn.Linear(128, int(input_size/64)*32),
            nn.ReLU(True),
            nn.Unflatten(dim=1, unflattened_size=(32, int(input_size/64))),
            #nn.MaxUnpool1d(4,4),
            nn.ConvTranspose1d(32, 16, 3, stride=4, padding=0, output_padding=1),
            nn.BatchNorm1d(16),
            #nn.MaxUnpool1d(4,4),
            nn.ConvTranspose1d(16, 8, 3, stride=4, padding=0, output_padding=1),
            nn.BatchNorm1d(8),
            nn.ReLU(True),
            #nn.MaxUnpool1d(4,4),
            nn.ConvTranspose1d(8, 1, 3, stride=4, 
            padding=0, output_padding=1)
            
        )

    def forward(self, x):
        encoder = self.encoder(x)
        x = self.decoder(encoder)
        return x, encoder

In [14]:
def save_score_distribution(model, data_loader, criterion, save_to=None, figsize=(8, 6), epoch=0):
    losses = []
    labels = []
    for (x_batch, y_batch) in data_loader:
        x_batch = x_batch.cuda()

        output = model(x_batch)
        loss = criterion(output, x_batch)

        loss = torch.mean(loss, dim=1)
        loss = loss.detach().cpu().numpy().flatten()
        losses.append(loss)

        labels.append(y_batch.detach().cpu().numpy().flatten())

    losses = np.concatenate(losses)
    labels = np.concatenate(labels)

    losses_0 = losses[labels == 0]
    losses_1 = losses[labels == 1]

    fig, ax = plt.subplots(1, figsize=figsize)

    ax.boxplot([losses_0, losses_1])
    ax.set_xticklabels(['normal', 'anomaly'])
    if epoch == 1 or epoch == 99:
        plt.show()
    plt.savefig(save_to)
    plt.close(fig)

In [15]:
# using difference between input & output we can get anomaly score
# (anomaly samples has higher difference between input and output then normal samples)
def get_difference_score(model, x, batch_size, extraction_type='melspectrogram'):

    dataset = TensorDataset(torch.tensor(x.astype(np.float32)))
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    predictions = []
    for (x_batch, ) in data_loader:
        x_batch = x_batch.cuda()
        preds = model(x_batch)
        predictions.append(preds[0].detach().cpu().numpy())

    predictions = np.concatenate(predictions)
    
    
    if extraction_type != 'melspectrogram':
        predictions = predictions.reshape(predictions.shape[0], predictions.shape[2])
        x=x.reshape(x.shape[0], x.shape[2])
        print(predictions.shape)
        print(x.shape)
        diff = ((x**2 - predictions**2)).mean(axis=1).reshape(-1, 1)
    else:
        
        
        diff = (x.mean(axis=2) - predictions.mean(axis=2)
                ).reshape(x.shape[0], x.shape[-1])

    return diff

In [16]:
def autoencoder_test(X_train, X_test, X_val, y_train, y_test, y_val, batch_size=64, lr=1e-3, epochs=10, extraction_type='aggregate_MFCC'):
    epochs = epochs
    input_size = X_test.shape[-1]
    model = Autoencoder(input_size).cuda()
    criterion = nn.MSELoss()
    per_sample_criterion = nn.MSELoss(reduction="none")
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)

    plot_path = r'plots'
    
    X_train =  X_train[:, np.newaxis, :]
    X_test = X_test[:, np.newaxis, :]
    X_val = X_val[:, np.newaxis, :]
    train_loader, test_loader = dataloader(
            X_train, y_train, X_test, y_test, batch_size)

    for epoch in range(epochs):
        running_loss = 0
        for (x_batch, _) in train_loader:
            x_batch = x_batch.cuda()

            output = model(x_batch)
            loss = criterion(output[0], x_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print("epoch [{}/{}], train loss:{:.4f}".format(epoch +
              1, epochs, running_loss))
        #if extraction_type != 'melspectrogram':
            #save_score_distribution(
               # model, test_loader, per_sample_criterion, plot_path, epoch=epoch)

    #test_score = get_difference_score(
    #    model, X_test, batch_size, extraction_type=extraction_type)
    test_score = get_difference_score(
        model, X_test, batch_size, extraction_type=extraction_type)


    # using classification algorithms we can classify samples by outier score (difference between input and output)
    score_forest = RandomForestClassifier(max_features=100, random_state=0)
    score_forest.fit(test_score, y_test)

    # Classification report on Validation data
    val_score = get_difference_score(
        model, X_val, batch_size, extraction_type=extraction_type)
    
    print(preproc.PyOD_classification_report(test_score, val_score, y_train,
                                             y_val, dataset='MIMII_DUE', extraction_type=extraction_type, contamination=contamination_mimii))

    
    prediction = score_forest.predict(val_score)
    accuracy = metrics.accuracy_score(y_val, prediction)
    precision = metrics.precision_score(y_val, prediction)
    recall = metrics.recall_score(y_val, prediction)
    f1_score = metrics.f1_score(y_val, prediction)
    scores = pd.DataFrame(
        [{'Extraction_type': extraction_type, 'Model_name': 'Autoencoder', 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1_score': f1_score}])

    return scores

In [17]:
# train-test-val from MIMII_DUE_ts dataset
X_train_mimii_ts, X_test_mimii_ts, X_val_mimii_ts, y_train_mimii, y_test_mimii, y_val_mimii = preproc.mix_data(
    [X_train_mimii_ts, X_test_mimii_ts], [y_train_mimii_ts, y_test_mimii_ts])

In [18]:
# MIMII_DUE Anomaly detection using Autoencoders (amplitude)
mimii_AE_ts = autoencoder_test(X_train_mimii_ts, X_test_mimii_ts, X_val_mimii_ts, y_train_mimii, y_test_mimii,
                                 y_val_mimii, batch_size=64, lr=1e-3, epochs=50, extraction_type='amplitude')
mimii_AE_ts

epoch [1/50], train loss:5.7605
epoch [2/50], train loss:3.0557
epoch [3/50], train loss:2.1364
epoch [4/50], train loss:1.6573
epoch [5/50], train loss:1.3285
epoch [6/50], train loss:1.0816
epoch [7/50], train loss:0.8943
epoch [8/50], train loss:0.7610
epoch [9/50], train loss:0.6662
epoch [10/50], train loss:0.5855
epoch [11/50], train loss:0.5178
epoch [12/50], train loss:0.4540
epoch [13/50], train loss:0.4084
epoch [14/50], train loss:0.3656
epoch [15/50], train loss:0.3381
epoch [16/50], train loss:0.3302
epoch [17/50], train loss:0.3096
epoch [18/50], train loss:0.2792
epoch [19/50], train loss:0.2579
epoch [20/50], train loss:0.2380
epoch [21/50], train loss:0.2264
epoch [22/50], train loss:0.2124
epoch [23/50], train loss:0.1999
epoch [24/50], train loss:0.1909
epoch [25/50], train loss:0.1800
epoch [26/50], train loss:0.3246
epoch [27/50], train loss:0.1914
epoch [28/50], train loss:0.1621
epoch [29/50], train loss:0.1496
epoch [30/50], train loss:0.1391
epoch [31/50], trai

,Extraction_type,Model_name,Accuracy,Precision,Recall,F1_score
0,amplitude,Autoencoder,0.917355,0.6,0.272727,0.375


In [19]:
# MIMII_DUE Anomaly detection using Autoencoders (amplitude)
toy_AE_ts = autoencoder_test(X_train_toy_ts, X_test_toy_ts, X_val_toy_ts, y_train_toy_ts, y_test_toy_ts,
                                 y_val_toy_ts, batch_size=64, lr=1e-3, epochs=50, extraction_type='amplitude')
toy_AE_ts

epoch [1/50], train loss:7.0474
epoch [2/50], train loss:2.1074
epoch [3/50], train loss:1.1781
epoch [4/50], train loss:0.6588
epoch [5/50], train loss:0.4030
epoch [6/50], train loss:0.2984
epoch [7/50], train loss:0.2484
epoch [8/50], train loss:0.2138
epoch [9/50], train loss:0.1824
epoch [10/50], train loss:0.1583
epoch [11/50], train loss:0.1374
epoch [12/50], train loss:0.1193
epoch [13/50], train loss:0.1037
epoch [14/50], train loss:0.0904
epoch [15/50], train loss:0.0796
epoch [16/50], train loss:0.0715
epoch [17/50], train loss:0.0637
epoch [18/50], train loss:0.0572
epoch [19/50], train loss:0.0526
epoch [20/50], train loss:0.0489
epoch [21/50], train loss:0.0456
epoch [22/50], train loss:0.0429
epoch [23/50], train loss:0.0409
epoch [24/50], train loss:0.0394
epoch [25/50], train loss:0.0380
epoch [26/50], train loss:0.0369
epoch [27/50], train loss:0.0360
epoch [28/50], train loss:0.0352
epoch [29/50], train loss:0.0345
epoch [30/50], train loss:0.0341
epoch [31/50], trai

,Extraction_type,Model_name,Accuracy,Precision,Recall,F1_score
0,amplitude,Autoencoder,0.797183,0.534351,0.864198,0.660377
